In [1]:
import numpy as np
import skimage
import imageio
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import skimage.feature as ft

import os
from os import path

import pandas as pd
import numpy as np

import tifffile
import glob
import cv2
import dask
import skimage as ski
import scipy.ndimage as ndi
#import functions as process
import skimage as ski


In [2]:
current_threshold = 2.2

In [3]:
folder_list = ['S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250512_TAL_24M6-4c_gridA2/maps/24M6-4c_gridA2/LayersData/Layer/ROI (2)/', 'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250506_TAL_24M6-4C_gridB1/maps/20250506_24M6-4C_gridB1/LayersData/Layer/ROI (3)/',
'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250508_TAL_24M6-10B_gridC2/maps/24M6-10B_gridC2/LayersData/Layer/ROI (3)/', 'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250512_TAL_24M6-4c_gridA2/maps/24M6-4c_gridA2/LayersData/Layer/ROI (3)/',
'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250508_TAL_24M6-10B_gridC2/maps/24M6-10B_gridC2/LayersData/Layer/ROI (2)/', 'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250530_TAL_24M6-4c_gridA2_oocyte1/LayersData/Layer/ROI (2)/', 'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250722_TAL_24M6-10b/24M6-10b_2/LayersData/Layer/ROI (2)/',
'S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250722_TAL_24M6-10b/24M6-10b_2/LayersData/Layer/ROI (3)/']

fnames = []
for folder in folder_list:
    print(folder)
    fnames.extend(glob.glob(folder + '*.tif'))

fnames.sort()
print(len(fnames))
fnames = fnames[0:1000]

S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250512_TAL_24M6-4c_gridA2/maps/24M6-4c_gridA2/LayersData/Layer/ROI (2)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250506_TAL_24M6-4C_gridB1/maps/20250506_24M6-4C_gridB1/LayersData/Layer/ROI (3)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250508_TAL_24M6-10B_gridC2/maps/24M6-10B_gridC2/LayersData/Layer/ROI (3)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250512_TAL_24M6-4c_gridA2/maps/24M6-4c_gridA2/LayersData/Layer/ROI (3)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250508_TAL_24M6-10B_gridC2/maps/24M6-10B_gridC2/LayersData/Layer/ROI (2)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250530_TAL_24M6-4c_gridA2_oocyte1/LayersData/Layer/ROI (2)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250722_TAL_24M6-10b/24M6-10b_2/LayersData/Layer/ROI (2)/
S:/micro/jeg/ag2588/mel/24M6_oocyte_ribosomes/20250722_TAL_24M6-10b/24M6-10b_2/LayersData/Layer/ROI (3)/
4414


In [4]:
def load_image(fname):
    rtn = ski.io.imread(fname)
    img = rtn
    img = img.astype(np.float32)
    min = np.percentile(img, .1)
    max = np.percentile(img, 99.9)
    img = (img - min) / (max - min)
    img = np.clip(img, 0, 1)
    return img

In [5]:
def process_file(fname, display=False, viewer=None, p0_thresh=current_threshold):
    # img = ski.io.imread(fname)[0:3770,:,0]
    # img = img.astype(np.float32)
    img = load_image(fname)
    img = cv2.GaussianBlur(img, (0, 0), 2)
    LoG0 = ndi.gaussian_laplace(img*1000.0, sigma=8)
    pts0 = ski.feature.peak_local_max(LoG0, min_distance=5, threshold_abs=p0_thresh)

    if display:
        viewer.layers.clear()
        viewer.add_image(img, name='img', contrast_limits=[0, 1])
        viewer.add_image(LoG0, name='LoG0', visible=False)
        viewer.add_points(pts0, name='pts0', size=20, face_color='blue')
    df = pd.DataFrame(pts0, columns=['y', 'x'])
    df['fname'] = fname
    df['intensity'] = LoG0[pts0[:,0], pts0[:,1]]
    df['p90'] = np.percentile(LoG0, 90)
    df['p99'] = np.percentile(LoG0, 99)
    df['p999'] = np.percentile(LoG0, 99.9)
    df['p10'] = np.percentile(LoG0, 10)
    return df

In [6]:
%%time
using_dask = True # If this is False, takes 12 minutes
all_data=[]

if using_dask:
    delayed = dask.delayed(process_file)
    for i, f in enumerate(fnames):
        all_data.append(delayed(f))
    all_data = dask.compute(*all_data)  
else:
    for i, f in enumerate(fnames):
        all_data.append(process_file(f))
        print([np.floor(i*100.0/len(fnames)), f])

CPU times: total: 18min 46s
Wall time: 1min 10s


In [7]:
df = pd.concat(all_data)
